# Image Preprocessing and Binary Classification with Keras

## Objective
In this week's exercise, you will:
1. Learn how to image preprocessing in keras.
2. Build and train a multilayer neural network for binary classification on a real-world dataset of cats and dogs.

---

## Step 1: Import Libraries
Let's start by importing the necessary libraries.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import os
import random
from tensorflow.keras import layers, models

---

## Step 2: Load and Preprocess the Data
We will use the Keras `ImageDataGenerator` for image augmentation and preprocessing.
First, unzip the uploaded dataset.


In [9]:
!unzip -q /content/kagglecatsanddogs_5340.zip

replace PetImages/Cat/0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [38]:
!ls

 CDLA-Permissive-2.0.pdf   kagglecatsanddogs_5340.zip   PetImages  'readme[1].txt'   sample_data


## Step 3: Learn about undersampling and implement it
Research online what undersampling and random undersampling is. It is a very powerful technique used often in machine Learning. Find out when it is used and undersample your dataset using "random undersampling"

In [ ]:
# undersample your dataset here
def undersample_data(directory, class_name, target_size):
  class_dir = os.path.join(directory, class_name) #path to the class foler
  images = os.listdir(class_dir) # listing all images in the class directory

  #Number of images in the class
  print(f"Total images in {class_name}:{len(images)}")

  #check if number of images is greater than or equal to the target size
  if len(images) < target_size:
    print(f"!! The class {class_name} has fewer images than {target_size}")
    return images #no undersampling done and return all the images

  # select a random subset of images
  undersampled_images = random.sample(images,target_size)

  #printing list of undersampled images
  print(f"Undersampled images from {class_name}")
  print(undersampled_images)

#set the path to unzipped dataset
train_dir = "/content/PetImages"

undersampled_dogs = undersample_data(train_dir, "Dog", 10)
undersampled_cats = undersample_data(train_dir, "Cat", 10)

---

## Step 4: Set Up ImageDataGenerator (or well more specifically the new version)
Were Sorry - the videos from the coursera course are sometimes not the most up to date. In this case the 'ImageDataGenerator' function is deprecated (look here https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) and will be removed in the future versions. The concept behind the new reccomended function is very similar though.
The new reccomendation is loading images with tf.keras.utils.image_dataset_from_directory and transforming the output tf.data.Dataset with preprocessing layers.

You may use Chat GPT for this task and you can also check the following tutorials <br>
https://www.tensorflow.org/tutorials/load_data/images <br>
https://www.tensorflow.org/tutorials/load_data/images <br>
https://www.tensorflow.org/guide/keras/preprocessing_layers <br>

In [ ]:
# TODO create a dataset using the recommended methods
# parameters for the loader
batch_size = 16
img_height = 128
img_width = 128

# loading of images
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels = "inferred", # infer the class labels based on the subfolder names
    label_mode = "int",
    batch_size = batch_size,
    image_size = (img_height, img_width),
    shuffle = True # shuffle the data set
)

# defining preprocessing layers

# data augmentation (artifically expanding trainig dataset) and normalisation
data_aug = tf.keras.Sequential([
  layers.RandomFlip("horizontal"),ok s
  layers.RandomRotation(0.2),
  #layers.RandomZoom(0.1)
])

normalization_layer = layers.Rescaling(1./255)

# Step 3: Apply preprocessing layers to the dataset
# Map data augmentation layer to the dataset
train_dataset = train_dataset.map(lambda x, y: (data_aug(x), y))

# Map normalization layer to the dataset
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))

# Step 4: Prefetch for performance
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

---

## Step 5: Build a Multilayer Neural Network
Now, let's build a multilayer neural network for binary classification.


In [22]:
# TODO build a model
model = models.Sequential([
    # First hidden layer with ReLU activation
    layers.Input(shape=(img_height, img_width, 3)),
    layers.Rescaling(1./255),  # Rescale the image to [0, 1] range

    layers.Conv2D(16, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    #layers.Conv2D(32, (3, 3), activation='relu'),
    #layers.MaxPooling2D((2, 2)),

    layers.Flatten(),

    layers.Dense(128, activation='relu'),

    # Output layer with 1 neuron for binary classification
    layers.Dense(1, activation='sigmoid')
])

model.summary()

# TODO compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ rescaling_6 (Rescaling)              │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 126, 126, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 63, 63, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 63504)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │       8,128,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,129,217 (31.01 MB)

 Trainable params: 8,129,217 (31.01 MB)

 Non-trainable params: 0 (0.00 B)

---

## Step 6: Train the Model
Train the model using the Dataset you created


In [23]:
model.fit(train_dataset, epochs = 5)

Epoch 1/5
 108/1563 ━━━━━━━━━━━━━━━━━━━━ 6:29 268ms/step - accuracy: 0.5005 - loss: 0.6939

InvalidArgumentError: Graph execution error:

Detected at node decode_image/DecodeImage defined at (most recent call last):
<stack traces unavailable>
Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]] [Op:__inference_one_step_on_iterator_10527]

---

## Step 7: Evaluate the Model
After training, you may upload some test images to evaluate your model.


In [21]:
from tensorflow.keras.preprocessing import image
import numpy as np
from google.colab import files

def load_and_predict(model):
    uploaded_files = files.upload()

    for fn in uploaded_files.keys():
        path = '/content/' + fn
        img = image.load_img(path, target_size=(150, 150))

        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0) / 255.0

        classes = model.predict(x)
        result = "a dog" if classes[0] > 0.5 else "a cat"

        print(f'The model predicts that the image is of {result}')

# Call the function to upload images and get predictions
load_and_predict(model)

Saving IMG_20190422_103302.jpg to IMG_20190422_103302.jpg


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_4" is incompatible with the layer: expected axis -1 of input shape to have value 28800, but received input with shape (1, 41472)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 150, 150, 3), dtype=float32)
  • training=False
  • mask=None